In [1]:
# General imports
import numpy as np
from deepmod_l1.analytical import theta_analytical

#Plotting imports
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Making library

In [2]:
x_sample = np.linspace(-5, 5, 50, dtype=np.float32)
t_sample = np.linspace(0, 5, 50, dtype=np.float32)
x_grid, t_grid = np.meshgrid(x_sample, t_sample, indexing='ij')
    
# Analytical
time_deriv, theta_noiseless = theta_analytical(x_grid, t_grid, 0.5, 0.25)

# V1; no convergence and selection

In [28]:
noise = np.var(time_deriv) * 1e-3
normalization = np.linalg.norm(theta_noiseless, axis=0)
theta = theta_noiseless / normalization
t = time_deriv + np.random.normal(scale= np.sqrt(noise), size=time_deriv.shape)

In [30]:
def initialize(theta, t):
    # Noise level
    beta = 1 / (np.var(t) * 0.1) # beta = 1/sigma^2
    
    # Finding best initial vector 
    projection = np.concatenate([((phi_i[:, None].T @ t).T @ (phi_i[:, None].T @ t)) / (phi_i[:, None].T @ phi_i[:, None]) for phi_i in theta.T])
    start_idx = np.argmax(projection)

    # Initializing alphas
    alfa = np.ones((theta.shape[1], 1)) * np.inf
    alfa[start_idx] = theta[:, start_idx:start_idx+1].T @ theta[:, start_idx:start_idx+1] / (projection[start_idx] - 1/beta)
    Phi = theta[:, [start_idx]]
    
    return Phi, alfa, beta

def posterior(Phi, t, alfa, beta):
    Sigma = np.linalg.inv(alfa[alfa != np.inf] * np.eye(Phi.shape[1]) + beta * Phi.T @ Phi)  # posterior covariance
    mu = beta * Sigma @ Phi.T @ t  # posterior mean
    
    return Sigma, mu

def sparse_quality_factor(theta, Phi, Sigma, alfa, beta):
    B = beta * np.eye(Phi.shape[0])
    precalc = B @ Phi @ Sigma @ Phi.T @ B

    Sm = np.concatenate([phi_i[:, None].T @ B @ phi_i[:, None] - phi_i[:, None].T @ precalc @ phi_i[:, None] for phi_i in theta.T])
    Qm = np.concatenate([phi_i[:, None].T @ B @ t - phi_i[:, None].T @ precalc @ t for phi_i in theta.T])

    sm = Sm/(1 - Sm/alfa)
    qm = Qm/(1 - Sm/alfa)
    
    return sm, qm

def update_design_matrix(theta, sm, qm, alfa):
    idx = np.random.choice(theta.shape[1])
    phi_i = theta[:, idx:idx+1]
    theta_i = qm[idx, 0]**2 - sm[idx, 0]
    
    # Decididing what to do 
    if (theta_i > 0) & (alfa[idx, 0] != np.inf):
        alfa[idx, 0] = sm[idx, 0]**2 / theta_i #reestimating
    elif (theta_i > 0) & (alfa[idx, 0] == np.inf):
        alfa[idx, 0] = sm[idx, 0]**2 / theta_i # adding alpha
    elif (theta_i< 0) & (alfa[idx, 0] != np.inf):
        alfa[idx, 0] = np.inf #removing alpha
    
    Phi = theta[:, alfa[:, 0] != np.inf] #rebuilding phi
        
    return Phi, alfa

def update_noise(Phi, t, mu, Sigma, alfa): 
    beta = (Phi.shape[0] - Phi.shape[1] + np.sum(alfa[alfa != np.inf] * np.diag(Sigma))) / ((t - Phi @ mu).T @ (t - Phi @ mu)) 
    return beta

In [31]:
# Initializing
Phi, alfa, beta = initialize(theta, t)
Sigma, mu = posterior(Phi, t, alfa, beta)
sm, qm = sparse_quality_factor(theta, Phi, Sigma, alfa, beta)

In [46]:
for it in np.arange(150):
    Phi, alfa = update_design_matrix(theta, sm, qm, alfa)
    Sigma, mu = posterior(Phi, t, alfa, beta)
    sm, qm = sparse_quality_factor(theta, Phi, Sigma, alfa, beta)
    beta = update_noise(Phi, t, mu, Sigma, alfa)
    if it % 50 == 0:
        print(alfa)
    

[[           inf]
 [           inf]
 [3.26772296e-03]
 [2.64097274e+03]
 [           inf]
 [           inf]
 [           inf]
 [           inf]
 [1.00321113e+03]]
[[           inf]
 [           inf]
 [3.26772297e-03]
 [2.64097274e+03]
 [           inf]
 [           inf]
 [           inf]
 [           inf]
 [1.00321113e+03]]
[[           inf]
 [           inf]
 [3.26772296e-03]
 [2.64097274e+03]
 [           inf]
 [           inf]
 [           inf]
 [           inf]
 [1.00321113e+03]]


In [47]:
1/beta

array([[0.0001187]])

In [68]:
noise

0.00012206738442182541

In [48]:
A = alfa[alfa != np.inf] * np.eye(Phi.shape[1]) + beta * Phi.T @ Phi

In [49]:
np.linalg.cond(A)

9.914873305538197

In [50]:
alfa

array([[           inf],
       [           inf],
       [3.26772296e-03],
       [2.64097274e+03],
       [           inf],
       [           inf],
       [           inf],
       [           inf],
       [1.00321113e+03]])

In [65]:
mu.T / normalization[alfa[:, 0] != np.inf]

array([[ 5.0069935e-01, -2.2107922e-03, -4.3951401e-04]])

In [67]:
qm**2 - sm

array([[-4.27172706e+03],
       [-8.01136850e+03],
       [ 2.83764020e+09],
       [ 2.21528893e+04],
       [-2.31155327e+03],
       [-5.77339929e+01],
       [-2.27447916e+01],
       [-3.37625071e+03],
       [ 5.80808428e+03]])

# Adding convergence

In [69]:
noise = np.var(time_deriv) * 1e-3
normalization = np.linalg.norm(theta_noiseless, axis=0)
theta = theta_noiseless / normalization
t = time_deriv + np.random.normal(scale= np.sqrt(noise), size=time_deriv.shape)

In [74]:
def initialize(theta, t):
    # Noise level
    beta = 1 / (np.var(t) * 0.1) # beta = 1/sigma^2
    
    # Finding best initial vector 
    projection = np.concatenate([((phi_i[:, None].T @ t).T @ (phi_i[:, None].T @ t)) / (phi_i[:, None].T @ phi_i[:, None]) for phi_i in theta.T])
    start_idx = np.argmax(projection)

    # Initializing alphas
    alfa = np.ones((theta.shape[1], 1)) * np.inf
    alfa[start_idx] = theta[:, start_idx:start_idx+1].T @ theta[:, start_idx:start_idx+1] / (projection[start_idx] - 1/beta)
    Phi = theta[:, [start_idx]]
    
    return Phi, alfa, beta

def posterior(Phi, t, alfa, beta):
    Sigma = np.linalg.inv(alfa[alfa != np.inf] * np.eye(Phi.shape[1]) + beta * Phi.T @ Phi)  # posterior covariance
    mu = beta * Sigma @ Phi.T @ t  # posterior mean
    
    return Sigma, mu

def sparse_quality_factor(theta, Phi, Sigma, alfa, beta):
    B = beta * np.eye(Phi.shape[0])
    precalc = B @ Phi @ Sigma @ Phi.T @ B

    Sm = np.concatenate([phi_i[:, None].T @ B @ phi_i[:, None] - phi_i[:, None].T @ precalc @ phi_i[:, None] for phi_i in theta.T])
    Qm = np.concatenate([phi_i[:, None].T @ B @ t - phi_i[:, None].T @ precalc @ t for phi_i in theta.T])

    sm = Sm/(1 - Sm/alfa)
    qm = Qm/(1 - Sm/alfa)
    
    return sm, qm

def update_design_matrix(theta, sm, qm, alfa):
    idx = np.random.choice(theta.shape[1])
    phi_i = theta[:, idx:idx+1]
    theta_i = qm[idx, 0]**2 - sm[idx, 0]
    
    # Decididing what to do 
    if (theta_i > 0) & (alfa[idx, 0] != np.inf):
        alfa[idx, 0] = sm[idx, 0]**2 / theta_i #reestimating
    elif (theta_i > 0) & (alfa[idx, 0] == np.inf):
        alfa[idx, 0] = sm[idx, 0]**2 / theta_i # adding alpha
    elif (theta_i< 0) & (alfa[idx, 0] != np.inf):
        alfa[idx, 0] = np.inf #removing alpha
    
    Phi = theta[:, alfa[:, 0] != np.inf] #rebuilding phi
        
    return Phi, alfa

def update_noise(Phi, t, mu, Sigma, alfa): 
    beta = (Phi.shape[0] - Phi.shape[1] + np.sum(alfa[alfa != np.inf] * np.diag(Sigma))) / ((t - Phi @ mu).T @ (t - Phi @ mu)) 
    return beta

def convergence(sm, qm, alfa):
    dt = qm**2 - sm
    delta_alfa= sm**2 / dt - alfa # check a_new - a
    converged = np.max(np.abs(delta_alfa[dt > 0])) < 10**-6 # if max delta_a < 10^-6 and all other dt < 0, it has converged
    
    return converged

In [75]:
# Initializing
Phi, alfa, beta = initialize(theta, t)
Sigma, mu = posterior(Phi, t, alfa, beta)
sm, qm = sparse_quality_factor(theta, Phi, Sigma, alfa, beta)

converged = False
while converged == False:
    Phi, alfa = update_design_matrix(theta, sm, qm, alfa)
    Sigma, mu = posterior(Phi, t, alfa, beta)
    sm, qm = sparse_quality_factor(theta, Phi, Sigma, alfa, beta)
    beta = update_noise(Phi, t, mu, Sigma, alfa)
    converged = convergence(sm, qm, alfa)

In [76]:
alfa

array([[           inf],
       [           inf],
       [3.26702711e-03],
       [7.02437222e+03],
       [3.97346431e+03],
       [           inf],
       [           inf],
       [           inf],
       [           inf]])

In [77]:
mu

array([[1.74953866e+01],
       [8.61199757e-03],
       [1.30597120e-02]])

In [78]:
qm**2 - sm

array([[-2.93805820e+03],
       [-2.80261667e+03],
       [ 1.92123330e+10],
       [ 8.30841866e+03],
       [ 1.75678116e+04],
       [-1.41402957e+03],
       [-2.88497653e+03],
       [-1.66577832e+03],
       [-1.86177632e+03]])

# Including choice

In [69]:
noise = np.var(time_deriv) * 0.1
normalization = np.linalg.norm(theta_noiseless, axis=0)
theta = theta_noiseless / normalization
t = time_deriv + np.random.normal(scale= np.sqrt(noise), size=time_deriv.shape)

In [70]:
def initialize(theta, t):
    # Noise level
    beta = 1 / (np.var(t) * 0.1) # beta = 1/sigma^2
    
    # Finding best initial vector 
    projection = np.concatenate([((phi_i[:, None].T @ t).T @ (phi_i[:, None].T @ t)) / (phi_i[:, None].T @ phi_i[:, None]) for phi_i in theta.T])
    start_idx = np.argmax(projection)

    # Initializing alphas
    alfa = np.ones((theta.shape[1], 1)) * np.inf
    alfa[start_idx] = theta[:, start_idx:start_idx+1].T @ theta[:, start_idx:start_idx+1] / (projection[start_idx] - 1/beta)
    Phi = theta[:, [start_idx]]
    
    return Phi, alfa, beta

def posterior(Phi, t, alfa, beta):
    Sigma = np.linalg.inv(alfa[alfa != np.inf] * np.eye(Phi.shape[1]) + beta * Phi.T @ Phi)  # posterior covariance
    mu = beta * Sigma @ Phi.T @ t  # posterior mean
    
    return Sigma, mu

def sparse_quality_factor(theta, Phi, Sigma, alfa, beta):
    B = beta * np.eye(Phi.shape[0])
    precalc = B @ Phi @ Sigma @ Phi.T @ B

    Sm = np.concatenate([phi_i[:, None].T @ B @ phi_i[:, None] - phi_i[:, None].T @ precalc @ phi_i[:, None] for phi_i in theta.T])
    Qm = np.concatenate([phi_i[:, None].T @ B @ t - phi_i[:, None].T @ precalc @ t for phi_i in theta.T])

    sm = Sm/(1 - Sm/alfa)
    qm = Qm/(1 - Sm/alfa)
    
    return sm, qm, Sm, Qm 

def update_design_matrix(theta, sm, qm, alfa, Sm, Qm):
    idx = optimal_vec(sm, qm, Sm, Qm, alfa)
    phi_i = theta[:, idx:idx+1]
    theta_i = qm[idx, 0]**2 - sm[idx, 0]
    
    # Decididing what to do 
    if (theta_i > 0) & (alfa[idx, 0] != np.inf):
        alfa[idx, 0] = sm[idx, 0]**2 / theta_i #reestimating
    elif (theta_i > 0) & (alfa[idx, 0] == np.inf):
        alfa[idx, 0] = sm[idx, 0]**2 / theta_i # adding alpha
    elif (theta_i< 0) & (alfa[idx, 0] != np.inf):
        alfa[idx, 0] = np.inf #removing alpha
    
    Phi = theta[:, alfa[:, 0] != np.inf] #rebuilding phi
        
    return Phi, alfa

def update_noise(Phi, t, mu, Sigma, alfa): 
    beta = (Phi.shape[0] - Phi.shape[1] + np.sum(alfa[alfa != np.inf] * np.diag(Sigma))) / ((t - Phi @ mu).T @ (t - Phi @ mu)) 
    return beta

def convergence(sm, qm, alfa):
    dt = qm**2 - sm
    delta_alfa= sm**2 / dt - alfa # check a_new - a
    converged = np.max(np.abs(delta_alfa[dt > 0])) < 10**-6 # if max delta_a < 10^-6 and all other dt < 0, it has converged
    
    return converged

def optimal_vec(sm, qm, Sm, Qm, alfa):
    basis_idx = alfa != np.inf  # idx of bases in model
    set_idx = alfa == np.inf  # idx of bases not in model
    
    add_basis = (Qm**2 - Sm)/Sm + np.log(Sm/Qm**2)
    del_basis = Qm**2/(Sm - alfa) - np.log(1-Sm/alfa)
    alfa_new = sm**2/(qm**2 - sm)
    redo_basis = Qm**2/(Sm + (1/alfa_new-1/alfa)**-1) - np.log(1 + Sm * (1/alfa_new-1/alfa))
    
    #Making everything into nice matrix
    add_basis[basis_idx] = np.nan
    dt = qm**2 - sm
    add_basis[dt <= 0] = np.nan #stuff above assumes dt > 0
    del_basis[set_idx] = np.nan
    redo_basis[set_idx] = np.nan
    
    # Deciding update
    possible_update = np.concatenate((add_basis, redo_basis, del_basis), axis=1)
  
    idx = np.unravel_index(np.nanargmax(possible_update), possible_update.shape)[0]
    
    return idx

In [71]:
# Initializing
Phi, alfa, beta = initialize(theta, t)
Sigma, mu = posterior(Phi, t, alfa, beta)
sm, qm, Sm, Qm = sparse_quality_factor(theta, Phi, Sigma, alfa, beta)

converged = False
while converged == False:
    Phi, alfa = update_design_matrix(theta, sm, qm, alfa, Sm, Qm)
    Sigma, mu = posterior(Phi, t, alfa, beta)
    sm, qm, Sm, Qm = sparse_quality_factor(theta, Phi, Sigma, alfa, beta)
    beta = update_noise(Phi, t, mu, Sigma, alfa)
    converged = convergence(sm, qm, alfa)

In [77]:
mu / normalization[alfa[:, 0] != np.inf][:, None]

array([[0.50114911],
       [0.00766854]])

array([[1220.6755 ,  265.13342],
       [ 265.13342,   57.58757]], dtype=float32)

In [78]:
Sigma

array([[0.0128321 , 0.00154697],
       [0.00154697, 0.00528659]])

In [50]:
alfa 

array([[       inf],
       [       inf],
       [0.00327703],
       [       inf],
       [       inf],
       [       inf],
       [       inf],
       [       inf],
       [       inf]])

In [51]:
1/beta

array([[0.00120296]])

In [52]:
noise

0.0012206738442182541

In [55]:
for idx in np.arange(10):
    noise = np.var(time_deriv) * 0.1
    normalization = np.linalg.norm(theta_noiseless, axis=0)
    theta = theta_noiseless / normalization
    t = time_deriv + np.random.normal(scale= np.sqrt(noise), size=time_deriv.shape)
    
    # Initializing
    Phi, alfa, beta = initialize(theta, t)
    Sigma, mu = posterior(Phi, t, alfa, beta)
    sm, qm, Sm, Qm = sparse_quality_factor(theta, Phi, Sigma, alfa, beta)

    converged = False
    while converged == False:
        Phi, alfa = update_design_matrix(theta, sm, qm, alfa, Sm, Qm)
        Sigma, mu = posterior(Phi, t, alfa, beta)
        sm, qm, Sm, Qm = sparse_quality_factor(theta, Phi, Sigma, alfa, beta)
        beta = update_noise(Phi, t, mu, Sigma, alfa)
        converged = convergence(sm, qm, alfa)
    print(mu.shape)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in log


(2, 1)
(2, 1)
(4, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(3, 1)


In [56]:
noise = np.var(time_deriv) * 0.05
normalization = np.linalg.norm(theta_noiseless, axis=0)
theta = theta_noiseless / normalization
t = time_deriv + np.random.normal(scale= np.sqrt(noise), size=time_deriv.shape)

for idx in np.arange(10):
    # Initializing
    Phi, alfa, beta = initialize(theta, t)
    Sigma, mu = posterior(Phi, t, alfa, beta)
    sm, qm, Sm, Qm = sparse_quality_factor(theta, Phi, Sigma, alfa, beta)

    converged = False
    while converged == False:
        Phi, alfa = update_design_matrix(theta, sm, qm, alfa, Sm, Qm)
        Sigma, mu = posterior(Phi, t, alfa, beta)
        sm, qm, Sm, Qm = sparse_quality_factor(theta, Phi, Sigma, alfa, beta)
        beta = update_noise(Phi, t, mu, Sigma, alfa)
        converged = convergence(sm, qm, alfa)
    print(mu)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in log


[[17.54688182]
 [ 0.0585946 ]
 [-0.16712367]]
[[17.54688182]
 [ 0.0585946 ]
 [-0.16712367]]
[[17.54688182]
 [ 0.0585946 ]
 [-0.16712367]]
[[17.54688182]
 [ 0.0585946 ]
 [-0.16712367]]
[[17.54688182]
 [ 0.0585946 ]
 [-0.16712367]]
[[17.54688182]
 [ 0.0585946 ]
 [-0.16712367]]
[[17.54688182]
 [ 0.0585946 ]
 [-0.16712367]]
[[17.54688182]
 [ 0.0585946 ]
 [-0.16712367]]
[[17.54688182]
 [ 0.0585946 ]
 [-0.16712367]]
[[17.54688182]
 [ 0.0585946 ]
 [-0.16712367]]


In [65]:
Sigma

array([[0.01297155]])

In [68]:
normalization @ normalization.T

9408.692

# Function for training

In [14]:
def initialize(theta, t):
    # Noise level
    beta = 1 / (np.var(t) * 0.1) # beta = 1/sigma^2
    
    # Finding best initial vector 
    projection = np.concatenate([((phi_i[:, None].T @ t).T @ (phi_i[:, None].T @ t)) / (phi_i[:, None].T @ phi_i[:, None]) for phi_i in theta.T])
    start_idx = np.argmax(projection)

    # Initializing alphas
    alfa = np.ones((theta.shape[1], 1)) * np.inf
    alfa[start_idx] = theta[:, start_idx:start_idx+1].T @ theta[:, start_idx:start_idx+1] / (projection[start_idx] - 1/beta)
    Phi = theta[:, [start_idx]]
    
    return Phi, alfa, beta

def posterior(Phi, t, alfa, beta):
    Sigma = np.linalg.inv(alfa[alfa != np.inf] * np.eye(Phi.shape[1]) + beta * Phi.T @ Phi)  # posterior covariance
    mu = beta * Sigma @ Phi.T @ t  # posterior mean
    
    return Sigma, mu

def sparse_quality_factor(theta, Phi, Sigma, alfa, beta):
    B = beta * np.eye(Phi.shape[0])
    precalc = B @ Phi @ Sigma @ Phi.T @ B

    Sm = np.concatenate([phi_i[:, None].T @ B @ phi_i[:, None] - phi_i[:, None].T @ precalc @ phi_i[:, None] for phi_i in theta.T])
    Qm = np.concatenate([phi_i[:, None].T @ B @ t - phi_i[:, None].T @ precalc @ t for phi_i in theta.T])

    sm = Sm/(1 - Sm/alfa)
    qm = Qm/(1 - Sm/alfa)
    
    return sm, qm, Sm, Qm 

def update_design_matrix(theta, sm, qm, alfa, Sm, Qm):
    idx = optimal_vec(sm, qm, Sm, Qm, alfa)
    phi_i = theta[:, idx:idx+1]
    theta_i = qm[idx, 0]**2 - sm[idx, 0]
    
    # Decididing what to do 
    if (theta_i > 0) & (alfa[idx, 0] != np.inf):
        alfa[idx, 0] = sm[idx, 0]**2 / theta_i #reestimating
    elif (theta_i > 0) & (alfa[idx, 0] == np.inf):
        alfa[idx, 0] = sm[idx, 0]**2 / theta_i # adding alpha
    elif (theta_i< 0) & (alfa[idx, 0] != np.inf):
        alfa[idx, 0] = np.inf #removing alpha
    
    Phi = theta[:, alfa[:, 0] != np.inf] #rebuilding phi
        
    return Phi, alfa

def update_noise(Phi, t, mu, Sigma, alfa): 
    beta = (Phi.shape[0] - Phi.shape[1] + np.sum(alfa[alfa != np.inf] * np.diag(Sigma))) / ((t - Phi @ mu).T @ (t - Phi @ mu)) 
    return beta

def convergence(sm, qm, alfa):
    dt = qm**2 - sm
    delta_alfa= sm**2 / dt - alfa # check a_new - a
    converged = np.max(np.abs(delta_alfa[dt > 0])) < 10**-6 # if max delta_a < 10^-6 and all other dt < 0, it has converged
    
    return converged

def optimal_vec(sm, qm, Sm, Qm, alfa):
    basis_idx = alfa != np.inf  # idx of bases in model
    set_idx = alfa == np.inf  # idx of bases not in model
    
    add_basis = (Qm**2 - Sm)/Sm + np.log(Sm/Qm**2)
    del_basis = Qm**2/(Sm - alfa) - np.log(1-Sm/alfa)
    alfa_new = sm**2/(qm**2 - sm)
    redo_basis = Qm**2/(Sm + (1/alfa_new-1/alfa)**-1) - np.log(1 + Sm * (1/alfa_new-1/alfa))
    
    #Making everything into nice matrix
    add_basis[basis_idx] = np.nan
    dt = qm**2 - sm
    add_basis[dt <= 0] = np.nan #stuff above assumes dt > 0
    del_basis[set_idx] = np.nan
    redo_basis[set_idx] = np.nan
    
    # Deciding update
    possible_update = np.concatenate((add_basis, redo_basis, del_basis), axis=1)
  
    idx = np.unravel_index(np.nanargmax(possible_update), possible_update.shape)[0]
    
    return idx

def SBL(theta, t):
    # Normalizing
    normalization = np.linalg.norm(theta, axis=0)
    theta_normalized = theta / normalization
    
    # Initializing
    Phi, alfa, beta = initialize(theta_normalized, t)
    Sigma, mu = posterior(Phi, t, alfa, beta)
    sm, qm, Sm, Qm = sparse_quality_factor(theta_normalized, Phi, Sigma, alfa, beta)
    
    # Running
    converged = False
    while converged == False:
        Phi, alfa = update_design_matrix(theta_normalized, sm, qm, alfa, Sm, Qm)
        Sigma, mu = posterior(Phi, t, alfa, beta)
        sm, qm, Sm, Qm = sparse_quality_factor(theta_normalized, Phi, Sigma, alfa, beta)
        beta = update_noise(Phi, t, mu, Sigma, alfa)
        converged = convergence(sm, qm, alfa)
    
    # Rescaling
    factor = normalization[alfa[:, 0] != np.inf][:, None] 
    mu = mu / factor
    Sigma = Sigma / (factor @ factor.T)
    
    
    return alfa, mu, Sigma, 1/beta

In [15]:
noise = np.var(time_deriv) * 0.1
normalization = np.linalg.norm(theta_noiseless, axis=0)
t = time_deriv + np.random.normal(scale= np.sqrt(noise), size=time_deriv.shape)

alfa, mu, Sigma, inferred_noise = SBL(theta_noiseless, t)

In [16]:
alfa

array([[           inf],
       [1.88373536e+01],
       [3.32177549e-03],
       [           inf],
       [           inf],
       [           inf],
       [           inf],
       [1.02419687e+01],
       [           inf]])

In [17]:
mu

array([[-0.02229136],
       [ 0.49659897],
       [ 0.03555032]])

In [18]:
Sigma

array([[ 1.87453256e-04,  8.90290495e-21, -1.19554210e-04],
       [ 7.96345568e-21,  1.01373825e-05, -1.13744648e-20],
       [-1.19554210e-04, -1.32621504e-20,  2.46095182e-04]])